In [2]:
pip install numpy==1.16.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 6.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.8.0 requires tensorboard-plugin-wit>=1.6.0, which is not installed.
tensorflow 2.8.0 requires h5py>=2.9.0, but you have h5py 2.8.0 which is incompatible.
tensorflow 2.8.0 requires numpy>=1.20, but you have numpy 1.16.6 which is incompatible.
mxnet 1.6.0 requires graphviz<0.9.0,>=0.8.1, but you have graphviz 0.16 which is incompatible.

[notice] A new release of pip available: 22.1.2 -> 23.0
[notice] To update, run: /Users/d/opt/miniconda3/envs/tensorflow/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import joblib

In [4]:
df=pd.read_csv("https://raw.githubusercontent.com/pplonski/datasets-for-start/master/adult/data.csv",skipinitialspace=True)

In [31]:
X_cols = [c for c in df.columns if c!= "income"]
X = df[X_cols]
Y = df["income"]

In [32]:
X.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


In [33]:
X.shape

(32561, 14)

In [34]:
Y.size

32561

In [35]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1234)

In [36]:
train_mode = dict(X_train.mode().iloc[0])
X_train = X_train.fillna(train_mode)
X_train

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
29700,37,Private,34146,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,68,United-States
1529,37,Private,26898,HS-grad,9,Divorced,Exec-managerial,Unmarried,White,Female,0,0,12,United-States
27477,26,Private,190762,Some-college,10,Married-civ-spouse,Sales,Husband,White,Male,0,0,18,United-States
31950,35,Private,189092,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States
4732,23,Private,260019,7th-8th,4,Never-married,Farming-fishing,Unmarried,Other,Male,0,0,36,Mexico
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27852,24,Private,223367,11th,7,Never-married,Craft-repair,Unmarried,White,Male,0,0,40,United-States
23605,20,Private,127185,Some-college,10,Never-married,Prof-specialty,Own-child,White,Female,0,0,15,United-States
1318,41,State-gov,144928,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States
25299,40,Private,199303,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States


In [37]:
train_mode

{'age': 31.0,
 'workclass': 'Private',
 'fnlwgt': 121124,
 'education': 'HS-grad',
 'education-num': 9.0,
 'marital-status': 'Married-civ-spouse',
 'occupation': 'Prof-specialty',
 'relationship': 'Husband',
 'race': 'White',
 'sex': 'Male',
 'capital-gain': 0.0,
 'capital-loss': 0.0,
 'hours-per-week': 40.0,
 'native-country': 'United-States'}

In [38]:
encoders = {}
for column in ['workclass', 'education', 'marital-status',
                'occupation', 'relationship', 'race',
                'sex','native-country']:
    categorical_convert = LabelEncoder()
    X_train[column] = categorical_convert.fit_transform(X_train[column])
    encoders[column]=categorical_convert

In [40]:
RF = RandomForestClassifier(n_estimators=100)
RF = RF.fit(X_train, Y_train)

In [41]:
ex = ExtraTreesClassifier(n_estimators=100)
ex = ex.fit(X_train,Y_train)


In [45]:
joblib.dump(train_mode,"./train_mode.joblib", compress=True)
joblib.dump(encoders, "./encoders.joblib",compress=True)
joblib.dump(RF, "./random_forest.joblib",compress=True)
joblib.dump(ex, "./extra_trees.joblib",compress=True)


['./extra_trees.joblib']